##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# LoggingTensorHook と StopAtStepHook を Keras コールバックに移行する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/logging_stop_hook">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/logging_stop_hook.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

TensorFlow 1 では、`tf.estimator.LoggingTensorHook` を使用してテンソルを監視および記録しますが、`tf.estimator.StopAtStepHook` は `tf.estimator.Estimator` でトレーニングする場合に指定されたステップでトレーニングを停止するのに役立ちます。このノートブックは、`Model.fit` でカスタム Keras コールバック（`tf.keras.callbacks.Callback`）を使用して、これらの API から TensorFlow 2 の同等のものに移行する方法を示しています。

Keras [コールバック](https://www.tensorflow.org/guide/keras/custom_callback)は、組み込みの Keras `Model.fit`/`Model.evaluate`/`Model.predict` API でのトレーニング/評価/予測中のさまざまな時点で呼び出されるオブジェクトです。`tf.keras.callbacks.Callback` API ドキュメントでコールバックの詳細を学ぶことができます。また、[独自のコールバックの作成](../..guide/keras/custom_callback.ipynb/)および[組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)（*コールバックの使用*セクション）ガイドも参照してください。 TensorFlow 1 の `SessionRunHook` から TensorFlow 2 の Keras コールバックへの移行については、[支援ロジック付きトレーニングの移行](sessionrunhook_callback.ipynb)ガイドをご覧ください。

## セットアップ

インポートとデモンストレーション用の単純なデータセットから始めます。

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]

# Define an input function.
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

## TensorFlow 1: テンソルをログに記録し、tf.estimator API でトレーニングを停止する

TensorFlow 1 では、さまざまなフックを定義してトレーニング動作を制御します。次に、これらのフックを `tf.estimator.EstimatorSpec` に渡します。

以下に例を示します。

- テンソル（モデルの重みや損失など）を監視/記録するには、`tf.estimator.LoggingTensorHook`（ `tf.train.LoggingTensorHook` はそのエイリアスです）を使用します。
- 特定のステップでトレーニングを停止するには、`tf.estimator.StopAtStepHook` を使用します（<br>`tf.train.StopAtStepHook` はそのエイリアスです）。

In [ ]:
def _model_fn(features, labels, mode):
  dense = tf1.layers.Dense(1)
  logits = dense(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())

  # Define the stop hook.
  stop_hook = tf1.train.StopAtStepHook(num_steps=2)

  # Access tensors to be logged by names.
  kernel_name = tf.identity(dense.weights[0])
  bias_name = tf.identity(dense.weights[1])
  logging_weight_hook = tf1.train.LoggingTensorHook(
      tensors=[kernel_name, bias_name],
      every_n_iter=1)
  # Log the training loss by the tensor object.
  logging_loss_hook = tf1.train.LoggingTensorHook(
      {'loss from LoggingTensorHook': loss},
      every_n_secs=3)

  # Pass all hooks to `EstimatorSpec`.
  return tf1.estimator.EstimatorSpec(mode,
                                     loss=loss,
                                     train_op=train_op,
                                     training_hooks=[stop_hook,
                                                     logging_weight_hook,
                                                     logging_loss_hook])

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
estimator.train(_input_fn)

## TensorFlow 2: カスタムコールバックと Model.fit を使用してテンソルをログに記録し、トレーニングを停止する

TensorFlow 2 では、組み込みの Keras `Model.fit`（または `Model.evaluate`）をトレーニング/評価に使用する場合、カスタム `tf.keras.callbacks.Callback` を定義することで、テンソルの監視とトレーニングの停止を構成できます。次に、それらを `Model.fit`（または`Model.evaluate`）の `callbacks` パラメータに渡します。 (詳細については、[独自のコールバックの作成](../..guide/keras/custom_callback.ipynb)ガイドを参照してください。)

以下に例を示します。

- `StopAtStepHook` の機能を再作成するには、`on_batch_end` メソッドをオーバーライドして特定のステップ数の後にトレーニングを停止するカスタムコールバック（以下では `StopAtStepCallback` という名前）を定義します。
- `LoggingTensorHook` の動作を再作成するには、名前によるテンソルへのアクセスがサポートされていないため、ログに記録されたテンソルを手動で記録および出力するカスタムコールバック（`LoggingTensorCallback`）を定義します。カスタムコールバック内にログ記録の頻度を実装することもできます。以下の例では、2 ステップごとに重みを出力します。 N 秒ごとにログを記録するなどの他のストラテジーも可能です。

In [ ]:
class StopAtStepCallback(tf.keras.callbacks.Callback):
  def __init__(self, stop_step=None):
    super().__init__()
    self._stop_step = stop_step

  def on_batch_end(self, batch, logs=None):
    if self.model.optimizer.iterations >= self._stop_step:
      self.model.stop_training = True
      print('\nstop training now')

class LoggingTensorCallback(tf.keras.callbacks.Callback):
  def __init__(self, every_n_iter):
      super().__init__()
      self._every_n_iter = every_n_iter
      self._log_count = every_n_iter

  def on_batch_end(self, batch, logs=None):
    if self._log_count > 0:
      self._log_count -= 1
      print("Logging Tensor Callback: dense/kernel:",
            model.layers[0].weights[0])
      print("Logging Tensor Callback: dense/bias:",
            model.layers[0].weights[1])
      print("Logging Tensor Callback loss:", logs["loss"])
    else:
      self._log_count -= self._every_n_iter

終了したら、新しいコールバック（`StopAtStepCallback` と `LoggingTensorCallback`）を `Model.fit` の `callbacks` パラメータに渡します。

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
model.compile(optimizer, "mse")

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
model.fit(dataset, callbacks=[StopAtStepCallback(stop_step=2),
                              LoggingTensorCallback(every_n_iter=2)])

## Next steps

コールバックの詳細については、次を参照してください。

- API ドキュメント: `tf.keras.callbacks.Callback`
- ガイド: [独自のコールバックの作成](../..guide/keras/custom_callback.ipynb/)
- ガイド: [組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)（*コールバックの使用*セクション）

次の移行関連のリソースも役立つ場合があります。

- [早期停止移行ガイド](early_stopping.ipynb): `tf.keras.callbacks.EarlyStopping` は組み込みの早期停止コールバックです
- [TensorBoard 移行ガイド](tensorboard.ipynb): TensorBoard により、指標の追跡と表示が可能になります
- [支援ロジック付きトレーニングの移行ガイド](sessionrunhook_callback.ipynb): TensorFlow 1 の `SessionRunHook` から TensorFlow 2 の Keras コールバックへ